# SIXT33N Project
## Phase 5: Final Integration - Speech

### EE 16B: Designing Information Devices and Systems II, Spring 2017


Written by Nathaniel Mailoa and Emily Naviasky (2016)

nmailoa@berkeley.edu &emsp; enaviasky@berkeley.edu

**Name 1**:

**Login**: ee16b-


**Name 2**:

**Login**: ee16b-

## Table of Contents

* [Introduction](#intro)
* [Launchpad Implementation of PCA](#PCA)
* [Integration](#part1)
* [Final Demo and Writeup](#part2)

<a id='intro'></a>
## Introduction

Now you are in your last steps! SIXT33N has slowly been taking shape and soon it will be running around causing trouble. All you need to do now is to put everything together in a single Launchpad sketch. The objective is to get the SIXT33N to move straight fast, move straight slow, turn right, or turn left when it hears the corresponding command.

The checkpoint for this phase is the demo of your project. Please read the last section of the notebook for details.

<a id='PCA'></a>
## <span style="color:blue">Launchpad Implementation of PCA</span>

### Materials
- Microphone front-end circuit
- Launchpad + USB

First, we will need to implement <b>data processing</b> and <b>classification</b> (just the projection, not the PCA) in the Launchpad sketch <b>`classify.ino`</b>. Quickly glance through <b>`classify.ino`</b>. It contains the same base code as the <b>`collect-data-envelope.ino`</b> from before, with some extra functionality. Since Energia does not have as many in-built functions as Python, you might have to write out the functions yourself. For example, a dot product should be written as:

`float result = 0;`<br/>
`for (i=0; i<LENGTH; i++){`<br/>
&emsp; `result += vector1[i]*vector2[i];`<br/>
`}`

There are 2 code blocks (A1 and B) that you need to modify. <b>You should not have to change anything else outside these marked code blocks.</b> 

First, fill in some constant parameters in **`CODE BLOCK A1`**. The `SNIPPET_SIZE`, `PRELENGTH` and `THRESHOLD` constants are found from the arguments to the enveloping function in the PCA classification phase. 

The `KMEANS_THRESHOLD` is the variable we suggest you use to decide whether to accept or regect a classification depending on a sample's distance from the closest centroid. If the L2 norm (distance) is larger than the threshold, your classification algorithm should simply ignore it and wait for the next sample. Look at the plot of sample data and the centroids from the PCA Notebook and approximate a radius around the centroids that capture most of the data. Try to be conservative - it's better to not classify than to misclassify.

The `LOUDNESS_THRESHOLD` variable decides whether to accept a classification depending on the amplitude of the recorded data. If the recorded data is too soft, we do not want to classify it as it is probably noise. Since the loudness unit is arbitrary, start by using `700`. Later, if the Launchpad classifies noise, increase this constant. If it misses a lot of speech (i.e. thinks your word is noise), decrease this constant.

Secondly, fill in the arrays in **`CODE BLOCK A3`**. These are the PCA bases and mean vector, as well as the K-means classification centroids. If you need more than 2 principal components, add a new `pca_vec3` array. Remember, if you are using more than two principal components, then the dimensionality of all of your point and the centriods has increased. The centroid arrays should be the same length as the number of principal components you use. An example of filling up an array of 3 rather than two elements is shown below for syntax.

<code>float centriod1[3] = {0.1234, 0.5678, 0.9012};</code>

Now go to **`CODE BLOCK B`**. This is the meat of the classification. Before this block, the call to `envelope` leaves the data vector in the array called `result`. You will write the code to project this data array onto your new PCA basis. Remember that you will still need to subtract the mean vector, before doing a dot product for each basis. We recommend using the variables `proj1` and `proj2` to store the projection results.

Next, code the K-means classification using the centroid arrays. Find the distance between the projected data point and each centroid using the function `l2_norm` (for 2 principal components) or `l2_norm3` (for 3 principal components). These functions take the data point coordinates and the centroid array. Look up the function definition in the sketch. Out of the 4 centroids, find the one with the smallest L2 norm from the data point. This is the word you classified.

Before saying you have classified a word, however, you will want to check the distance first. If the distance is greater than `KMEANS_THRESHOLD`, the data point is not close enough for you to guess with some certainty. Thus, we only consider a data point to be classified if the distance is less than this constant.

When you have classified a data point, print out the word on Serial. You should be able to see the printout from Energia's Serial Monitor.

Before running the code in the Launchpad, use the oscilloscope to probe the circuit output again and make sure that it still averages around 1.65V. Now upload the sketch to your Launchpad and run it. Open the Serial Monitor and press the reset button. Say your word and the Launchpad should recognize your word as it gets printed on the Serial Monitor!

If the Launchpad does not classify as well as you think it should, remember to play with the `KMEANS_THRESHOLD` and `LOUDNESS_THRESHOLD` variables. To debug the sketch, you can also print out any of the variables you have used. 

Voila! Your SIXT33N can recognize your words!

<a id='part1'></a>
## <span style="color:blue">Integration</span>

Everything you work on in this phase will be in the sketch `integration.ino`, which is your final sketch for the SIXT33N. Open the sketch - we will quickly walk through the sketch. As we go through it, fill in sections labelled `YOUR CODE HERE` as instructed.


### Encoder Pins

The top of the sketch contains some constant and variable definitions. We start with defining pins. The choice of these pins (especially the encoder pins) are embedded in the actual code, so if pins 2.5 or 1.2 got burned at some point in the semester, now is the time to flag down your GSI. They will help you modify the important part of the sketch.

### Enveloping and PCA Vectors

In the main body of the sketch, the code defines 2 modes: `MODE_LISTEN` and `MODE_DRIVE`. We can look at the 2 modes and see what they are doing. 

In `MODE_LISTEN`, the SIXT33N listens to your words and tries to classify the word. The Launchpad will repeatedly record some samples and perform the enveloping. If the Launchpad sucessfully classifies a word then it will enter `MODE_DRIVE`.

First, look for **`CODE BLOCK A1`** near the top of the sketch. Copy the same code block A1 from the `classify.ino` sketch from the processing integration phase. This should provide the Launchpad with the snippet size, pre-length, enveloping threshold, K-means threshold and loudness threshold.

Next, look for **`CODE BLOCK A3`**. Copy the same code block A3 from the `classify.ino` sketch from the processing integration phase. This should provide the Launchpad with the PCA vector, mean vector as well as the K-means centroids.

Note: you may find that you encounter an "out-of-memory" error - this is because our Launchpads do not have a ton of memory. Try reducing your SNIPPET_SIZE (window length) so your vectors aren't quite as long. This may change your classification, but hopefully for the better.


### Wheel Control 

In `MODE_DRIVE`, the SIXT33N executes the closed loop control. Depending on the word that was classified, it will set a desired speed and then go straight at that speed.

Copy your $\theta$, $\beta$, and k values from `closed_loop.ino` into the block labeled **`CODE BLOCK A2`**. This should tell the Launchpad the gain values of the control loop and add new variables as needed to turn. Next, copy the required code from `CODE BLOCK C`.


### Classification and Driving

Next, go to the section labeled **`CODE BLOCK B`** in the `loop()` function. Copy the code from code block B in `classify.ino` from the processing integration phase here. This block should do the PCA projection and classification and identify a word. 

Then, based on the identified commands, write code that makes the SIXT33N behave as desired. As a reminder, the four commands should make the SIXT33N:
- Drive straight fast
- Drive straight slow
- Turn left
- Turn right

You might want to look back at the `turning.ino` sketch to see how you implement the turns. You can modify any of the CODE BLOCKs to implement the turns.

(Hint: look to CODE BLOCK D from `turning.ino`)

If you changed the **`INITIAL_PWM`** constant near the top of the sketch in the controls phase, you also need to modify it in `integration.ino`.


That's it! You should be set. Now load the sketch into your Launchpad and make sure your Launchpad's 5V pin is connected to the voltage regulator output. If you forgot the pins you should attach to the motors and encoders, you can find them in the define statements in the top of the Launchpad sketch. If your car doesn't have as much drive as before, you might need to change to fresh batteries - check the voltage of your battery using the Digital Multimeter and if it is considerably less than 9V ask for fresh ones.

Put your car on the floor and try shouting some commands at it! Hopefully, it can classify both commands correctly and move in a straight line at different speeds.

Did it work? If not, you might still need to tweak your control parameters. If it did work: time to celebrate!

<a id='part2'></a>
## <span style="color:blue">Final Demo and Writeup</span>

Congratulations - You're done! If you have some time, be creative and try to add a functionality to the SIXT33N! Don't hesitate to ask for your GSI's help when you want to modify the Launchpad sketch since it is more complicated than previous sketches. 

### Demo

You have the option to either demo the project during your lab time in the RRR week or to take a video of it working properly. In the demo, you will have to:
- Set SIXT33N on the ground, then say the commands one-by-one. You should at least say each command twice, in any order you'd like.
- You are allowed to reposition the car inbetween the steps if you have trouble crashing into walls.

#### Live Demo

During the live demo, you will bring your SIXT33N and show the GSIs your project. Say the 4 different words it recognizes and show that the SIXT33N turns or moves straight according to the desired behavior.

#### Video Demo

Instead of doing a live demo, you can also record a video of your project and upload it to YouTube.
- The video has to include the members' faces. Start by introducing your group. Also explain what are the commands you will use and what are the desired behavior of each command.
- The video cannot be edited or sliced - the demo should be continuous.
- Email your TA the link to the uploaded video by <b>Friday, April 28th at 10 PM</b>. NO LATE SUBMISSIONS ARE ACCEPTED.

See [here](https://inst.eecs.berkeley.edu/~ee16b/sp17/proj/proj-deliv.pdf) for more detailed instructions for your demo.


### Writeup

On top of the demo, you will need to submit a writeup of the project. Briefly summarize each phase of your project and elaborate on your design choices. The writeup will be around 2 pages uploaded to glookup in the assignment proj_writeup as <b>proj_writeup.pdf</b>, due by <b>Friday, April 28th at 10 PM</b>. NO LATE SUBMISSIONS ARE ACCEPTED.

The writeup should hit these topics:
- Circuit
- PCA Classification
- Control
- General
- If you did a video chekoff, include a link to your video

See [here](https://inst.eecs.berkeley.edu/~ee16b/sp17/proj/proj-deliv.pdf) for more detailed instructions for your report.